In [2]:
import cv2
import os
import math

def split_video_with_sliding_window(video_path, output_folder, window_size, stride):
    """
    スライディングウィンドウ法を用いて動画を分割し、ファイルに出力する関数。

    Args:
        video_path (str): 入力動画のパス。
        output_folder (str): 分割後の動画を保存するフォルダのパス。
        window_size (int): ウィンドウサイズ（フレーム数）。
        stride (int): ストライド（フレーム数）。
    """
    # --- 1. 事前準備 ---
    # 出力フォルダがなければ作成
    os.makedirs(output_folder, exist_ok=True)
    
    # 動画を読み込むためのオブジェクトを作成
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"エラー: 動画ファイルが開けませんでした: {video_path}")
        return

    # 動画のプロパティを取得
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"入力動画: {os.path.basename(video_path)}")
    print(f"  - FPS: {fps}, 総フレーム数: {total_frames}")
    print(f"  - ウィンドウサイズ: {window_size} フレーム, ストライド: {stride} フレーム")
    print("-" * 50)
    
    # --- 2. 全フレームをリストに読み込む ---
    all_frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        all_frames.append(frame)
    cap.release()
    
    # --- 3. スライディングウィンドウで動画を分割・保存 ---
    clip_count = 0
    # ウィンドウの開始位置をストライドごとにずらしながらループ
    for start_frame in range(0, total_frames - window_size + 1, stride):
        end_frame = start_frame + window_size
        
        # 現在のウィンドウに対応するフレームのサブセットを取得
        clip_frames = all_frames[start_frame:end_frame]
        
        # 出力ファイル名を設定
        output_filename = f"clip_{clip_count + 1:03d}.mp4"
        output_path = os.path.join(output_folder, output_filename)
        
        # 動画書き出し用のオブジェクトを作成
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # フレームを1枚ずつ書き出す
        for frame in clip_frames:
            out.write(frame)
        
        out.release()
        print(f"✅ 保存完了: {output_filename} (フレーム {start_frame} - {end_frame - 1})")
        clip_count += 1
        
    print("-" * 50)
    print(f"🎉 全ての処理が完了しました。合計 {clip_count}個のクリップが作成されました。")


# --- ここから実行 ---
if __name__ == "__main__":
    # --- 設定項目 ---
    # 1. 元となる動画ファイルのパスを指定
    INPUT_VIDEO_PATH = r"C:\Users\admin\Downloads\example1.mp4" # あなたの動画ファイルのパスに変更してください

    # 2. 分割した動画を保存するフォルダのパスを指定
    OUTPUT_FOLDER_PATH = r"C:\Users\admin\Downloads\sw" # 出力先フォルダのパスに変更してください
    
    # 3. ウィンドウサイズ（1クリップあたりのフレーム数）
    WINDOW_SIZE_FRAMES = 30  # 例: 30fpsの動画なら2秒分
    
    # 4. ストライド（次のクリップを何フレームずらして始めるか）
    STRIDE_FRAMES = 30  # 例: 30fpsの動画なら1秒ずつずらす
    # ------------------
    
    if not os.path.exists(INPUT_VIDEO_PATH):
        print(f"エラー: 指定された動画ファイルが見つかりません: {INPUT_VIDEO_PATH}")
    else:
        split_video_with_sliding_window(
            video_path=INPUT_VIDEO_PATH,
            output_folder=OUTPUT_FOLDER_PATH,
            window_size=WINDOW_SIZE_FRAMES,
            stride=STRIDE_FRAMES
        )

入力動画: example1.mp4
  - FPS: 30.004774629128587, 総フレーム数: 199
  - ウィンドウサイズ: 30 フレーム, ストライド: 30 フレーム
--------------------------------------------------
✅ 保存完了: clip_001.mp4 (フレーム 0 - 29)
✅ 保存完了: clip_002.mp4 (フレーム 30 - 59)
✅ 保存完了: clip_003.mp4 (フレーム 60 - 89)
✅ 保存完了: clip_004.mp4 (フレーム 90 - 119)
✅ 保存完了: clip_005.mp4 (フレーム 120 - 149)
✅ 保存完了: clip_006.mp4 (フレーム 150 - 179)
--------------------------------------------------
🎉 全ての処理が完了しました。合計 6個のクリップが作成されました。
